In [1]:
!nvidia-smi

Sun Aug  6 19:14:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.05              Driver Version: 535.86.05    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8               1W /  50W |      5MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/CrowdAnalysis/WalkInWalkOutCounter


In [3]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.143 🚀 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)
Setup complete ✅ (8 CPUs, 15.5 GB RAM, 44.0/72.3 GB disk)


In [4]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80

/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/CrowdAnalysis/WalkInWalkOutCounter
fatal: destination path 'ByteTrack' already exists and is not an empty directory.
/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/CrowdAnalysis/WalkInWalkOutCounter/ByteTrack


In [5]:
!pip3 install -q -r requirements.txt
!pip install -e .
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop
!pip install numpy==1.22.4

from IPython import display
display.clear_output()

In [6]:
import sys
sys.path.append(f"{HOME}/ByteTrack")
import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [7]:
!pip install supervision==0.2.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.2.0


In [8]:
!pip install supervision==0.2.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

supervision.__version__: 0.2.0


## Install Roboflow Supervision

In [9]:
from supervision.draw.color import ColorPalette
from supervision import Point
from supervision import VideoInfo
from supervision import get_video_frames_generator
from supervision import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision import Detections, BoxAnnotator
from supervision.detection.line_counter import LineZone, LineZoneAnnotator

## Tracking utils

Unfortunately, we have to manually match the bounding boxes coming from our model with those created by the tracker.

In [10]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

## Load pre-trained YOLOv8 model

In [11]:
# settings
%cd {HOME}

/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/CrowdAnalysis/WalkInWalkOutCounter


## Predict and annotate single frame

In [13]:
import supervision as sv
import cv2
from ultralytics import YOLO

vid = "/home/manz/Desktop/My Stuffs/Work Stuffs/Python Envs/HyperSpace/ComputerVision/Data/Crowd_Count/Sample_Videos/JapanStreet.mp4"
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
model.fuse()

# extract video frame
generator = sv.get_video_frames_generator(vid)
iterator = iter(generator)
frame = next(iterator)

# detect
results = model.predict(frame, imgsz=1280)[0]
detections = sv.Detections.from_yolov8(results)
detections = detections[detections.class_id==0]

# annotate
box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=2, text_scale=1, text_padding = 1)
labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, confidence, class_id, _ in detections]
frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

%matplotlib inline
# sv.show_frame_in_notebook(frame, (10, 10))
def mouse_callback(event, x, y, flags, param):
    global click_count, click_points

    if event == cv2.EVENT_LBUTTONDOWN and click_count < 2:
        click_points.append((x, y))
        click_count += 1
        cv2.circle(image, (x, y), 5, (0, 255, 255), -1)  # Yellow color (BGR)

# Initialize variables
click_count = 0
click_points = []

# Load the image
image = frame

# Open a window to display the image
cv2.namedWindow('Click Points')
for i in range(2):
    cv2.imshow('Click Points', image,)
cv2.setMouseCallback('Click Points', mouse_callback)

# Wait for user clicks and display the points
while click_count < 2:
    key = cv2.waitKey(10)
    if key == 27:  # Exit loop if the 'Esc' key is pressed
        break

# Close the image window
cv2.destroyAllWindows()

# Extract click point values
x1, y1 = click_points[0]
x2, y2 = click_points[1]

# Print the values
print("x1:", x1, "y1:", y1)
print("x2:", x2, "y2:", y2)

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

0: 736x1280 14 persons, 1 bicycle, 4 backpacks, 6 handbags, 273.3ms
Speed: 4.3ms preprocess, 273.3ms inference, 0.9ms postprocess per image at shape (1, 3, 736, 1280)


x1: 633 y1: 577
x2: 1752 y2: 961


## Predict and annotate whole video

In [18]:
VideoInfo.from_video_path(vid)

VideoInfo(width=1920, height=1080, fps=30, total_frames=3969)

In [23]:
import cv2

LINE_START = Point(x1,y1)
LINE_END = Point(x2,y2)
# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(vid)
# create frame generator
generator = get_video_frames_generator(vid)
# create LineCounter instance
line_counter = LineZone(start=LINE_START, end=LINE_END)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(thickness=2, text_thickness=2, text_scale=1, text_padding = 1)
line_annotator = LineZoneAnnotator(thickness=3, text_thickness=2, text_scale=3, text_padding=1)

# open target video file
# loop over video frames
for frame in generator:
    # model prediction on single frame and conversion to supervision Detections
    results = model.predict(frame, imgsz=1280)[0]
    detections = sv.Detections.from_yolov8(results)

    # filtering out detections with unwanted classes
    detections = detections[detections.class_id==0]

    # tracking detections
    tracks = byte_tracker.update(
        output_results=detections2boxes(detections=detections),
        img_info=frame.shape,
        img_size=frame.shape
    )
    tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)

    detections.tracker_id = np.array(tracker_id)

    # filtering out detections without trackers
    mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
    detections.filter(mask=mask, inplace=True)

    # format custom labels
    labels = [
        f"#{tracker_id} -- {confidence:0.2f}"
        for _, confidence, class_id, tracker_id
        in detections
    ]

    # updating line counter
    line_counter.trigger(detections=detections)

    # annotate and display frame
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    line_annotator.annotate(frame=frame, line_counter=line_counter)
    
    original_height, original_width = image.shape[:2]
    desired_height = int(original_height * 0.7)
    desired_width = int(original_width * 0.7)
    small_image = cv2.resize(frame, (desired_width, desired_height))

    
    
    cv2.imshow('Sample_Output', small_image)
    # result.write(frame)
    # if detection.__len__() == 15:
    #     breakq
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


0: 736x1280 14 persons, 1 bicycle, 4 backpacks, 6 handbags, 276.5ms
Speed: 21.6ms preprocess, 276.5ms inference, 0.9ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 13 persons, 1 bicycle, 3 backpacks, 5 handbags, 1 suitcase, 274.8ms
Speed: 5.9ms preprocess, 274.8ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 14 persons, 1 bicycle, 6 backpacks, 4 handbags, 1 suitcase, 275.4ms
Speed: 5.2ms preprocess, 275.4ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 13 persons, 1 bicycle, 4 backpacks, 3 handbags, 1 suitcase, 278.7ms
Speed: 5.3ms preprocess, 278.7ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 13 persons, 1 bicycle, 5 backpacks, 4 handbags, 274.8ms
Speed: 5.6ms preprocess, 274.8ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 14 persons, 1 bicycle, 4 backpacks, 4 handbags, 275.1ms
Speed: 5.3ms preprocess, 275.1ms inference, 1.1ms post

In [13]:
import supervision as sv
import cv2
from ultralytics import YOLO

cam = '/dev/video0'
vid = cv2.VideoCapture(cam)
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
model.fuse()
ret, frame = vid.read()


%matplotlib inline
# sv.show_frame_in_notebook(frame, (10, 10))
def mouse_callback(event, x, y, flags, param):
    global click_count, click_points

    if event == cv2.EVENT_LBUTTONDOWN and click_count < 2:
        click_points.append((x, y))
        click_count += 1
        cv2.circle(image, (x, y), 5, (0, 255, 255), -1)  # Yellow color (BGR)

# Initialize variables
click_count = 0
click_points = []

# Load the image
image = frame

# Open a window to display the image
cv2.namedWindow('Click Points')
for i in range(2):
    cv2.imshow('Click Points', image,)
cv2.setMouseCallback('Click Points', mouse_callback)

# Wait for user clicks and display the points
while click_count < 2:
    key = cv2.waitKey(10)
    if key == 27:  # Exit loop if the 'Esc' key is pressed
        break

# Close the image window
cv2.destroyAllWindows()
vid.release()
# Extract click point values
x1, y1 = click_points[0]
x2, y2 = click_points[1]

# Print the values
print("x1:", x1, "y1:", y1)
print("x2:", x2, "y2:", y2)

LINE_START = Point(x1,y1)
LINE_END = Point(x2,y2)

# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())

# create LineCounter instance
line_counter = LineZone(start=LINE_START, end=LINE_END)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(thickness=2, text_thickness=2, text_scale=1, text_padding = 1)
line_annotator = LineZoneAnnotator(thickness=2, text_thickness=2, text_scale=1, text_padding=1)

# open target video file
# loop over video frames
cap = cv2.VideoCapture(cam)
# Check if the camera is opened
if not cap.isOpened():
    cap = cv2.VideoCapture(cam)


while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        break
    
    results = model.predict(frame, imgsz=1280)[0]
    detections = sv.Detections.from_yolov8(results)

    # filtering out detections with unwanted classes
    # detections = detections[detections.class_id==0]

    # tracking detections
    tracks = byte_tracker.update(
        output_results=detections2boxes(detections=detections),
        img_info=frame.shape,
        img_size=frame.shape
    )
    tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)

    detections.tracker_id = np.array(tracker_id)

    # filtering out detections without trackers
    mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
    detections.filter(mask=mask, inplace=True)

    # format custom labels
    labels = [
        f"#{tracker_id} -- {confidence:0.2f}"
        for _, confidence, class_id, tracker_id
        in detections
    ]

    # updating line counter
    line_counter.trigger(detections=detections)

    # annotate and display frame
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    line_annotator.annotate(frame=frame, line_counter=line_counter)
    
    cv2.imshow('Sample_Output', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


error: OpenCV(4.8.0) /io/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [ ]:
x = cv2.VideoCapture

In [2]:
import supervision as sv

sv.VideoInfo.from_video_path('/dev/Video0')

Exception: Could not open video at /dev/Video0